#**Setup**

---



In [ ]:

!git clone https://github.com/ai4bharat/indic-bert
%cd indic-bert
!pip3 install -r requirements.txt
%cd ..
!mkdir indic-glue outputs

#**Download Datasets**
---


In [ ]:

% cd indic-glue
# Download the dataset -- insert link obtained from https://indicnlp.ai4bharat.org/indic-glue/#downloads
!wget https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/evaluations/wiki-cloze.tar.gz
!tar -xaf wiki-cloze.tar.gz
% cd ..


#**Fine-tune the Model**
---



In [ ]:

%cd indic-bert

import os

from fine_tune.cli import main as finetune_main

argvec = ['--lang', 'gu',
          '--dataset', 'wiki-cloze', # use the right dataset key, check https://github.com/AI4Bharat/indic-bert/blob/master/fine_tune/cli.py#L10
          '--model', 'ai4bharat/indic-bert',
          '--iglue_dir', '../indic-glue',
          '--output_dir', '../outputs',
          '--max_seq_length', '128',
          '--learning_rate', '2e-5',
          '--num_train_epochs', '3',
          '--train_batch_size', '32'
]

finetune_main(argvec)

#**Check the Results**
---

In [ ]:
!cat /content/outputs/wiki-cloze/gu-gu/model-ai4bharat-indic-bert/test_results.txt